In [ ]:
GITHUB_USERNAME = "$GITHUB_USERNAME$"
GITHUB_REF = "$GITHUB_REF$"
NOTEBOOK_TYPE = "$NOTEBOOK_TYPE$"
PYTHON_VERSION = "$PYTHON_VERSION$"
IPYTHON_VERSION = "$IPYTHON_VERSION$"

In [ ]:
from pathlib import Path

import requests


if NOTEBOOK_TYPE == 'colab':
    # utils module doesn't exist on colab VM, so get current version from GitHub
    utils_module = Path('utils.py').resolve()
    response = requests.get(f'https://raw.githubusercontent.com/{GITHUB_USERNAME}/davos/{GITHUB_REF}/tests/utils.py')
    utils_module.write_text(response.text)
    # also need to install davos locally
    from utils import install_davos
    install_davos(source='github', ref=GITHUB_REF, fork=GITHUB_USERNAME)

In [ ]:
import json
import subprocess
import sys

import pkg_resources

from utils import mark, raises, run_tests, TestingEnvironmentError

In [ ]:
IPYTHON_SHELL = get_ipython()

# tests for general testing environment & package initialization
tests GitHub runner itself, as well as contents of `__init__.py` & `implementations.__init__.py`

In [5]:
def test_import_davos():
    global davos
    import davos

In [ ]:
def test_expected_python_version():
    installed_version = '.'.join(map(str, sys.version_info[:2]))
    expected_version = PYTHON_VERSION
    if installed_version != expected_version:
        raise TestingEnvironmentError(
            f"Test environment has Python {sys.version.split()[0]}, expected "
            "{PYTHON_VERSION}"
        )

In [ ]:
@mark.jupyter
def test_notebook_using_kernel_python():
    if not sys.executable.endswith('envs/kernel-env/bin/python'):
        raise TestingEnvironmentError(
            "Notebook does not appear to be using the correct python "
            "executable. Expected a path ending in "
            f"'envs/kernel-env/bin/python', found {sys.executable}"
        )

In [ ]:
@mark.skipif(IPYTHON_VERSION == 'latest', reason="runs when IPYTHON_VERSION != 'latest'")
def test_expected_ipython_version():
    try:
        pkg_resources.get_distribution(f"IPython=={IPYTHON_VERSION}")
    except pkg_resources.VersionConflict as e:
        raise TestingEnvironmentError(
            f"Test environment has IPython=={IPython.__version__}, expected "
            f"{IPYTHON_VERSION}") from e

In [ ]:
@mark.skipif(IPYTHON_VERSION != 'latest', reason="runs when IPYTHON_VERSION == 'latest'")
def test_latest_ipython_version():
    pip_exe = davos.config.pip_executable
    outdated_pkgs = subprocess.check_output(
            [pip_exe, 'list', '--outdated', '--format', 'json'], encoding='utf-8'
        )
    outdated_pkgs_json = json.loads(outdated_pkgs)
    for pkg in outdated_pkgs_json:
        if pkg['name'] == 'ipython':
            raise TestingEnvironmentError(
                f"Test environment has IPython=={pkg['version']}, expected "
                f"latest version (IPython=={pkg['latest_version']})"
            )

In [ ]:
@mark.colab
def test_ipython_shell_is_colab_shell():
    if not isinstance(IPYTHON_SHELL, google.colab._shell.Shell):
        google_version = pkg_resources.get_distribution('google').version
        raise TestingEnvironmentError(
            "Qualified name for Colab interactive shell class has changed "
            "(google module may have been recently updated).\n\tShell type:\t"
            f"{type(IPYTHON_SHELL)}\n\tgoogle version:\t{google_version}"
        )

In [ ]:
def test_smuggle_in_namespace():
    assert 'smuggle' in globals()
    assert 'smuggle' in IPYTHON_SHELL.user_ns
    assert globals()['smuggle'] is IPYTHON_SHELL.user_ns['smuggle']

In [ ]:
def test_activated_on_import():
    assert davos.is_active()

In [ ]:
def test_deactivate_reactivate_toplevel():
    assert davos.is_active()
    
    davos.deactivate()
    assert not davos.is_active()
    
    with raises(NameError, match="name 'smuggle' is not defined"):
        smuggle ast
        
    davos.activate()
    assert davos.is_active()

In [ ]:
def test_all_configurable_fields_settable_via_configure():
    all_properties = []
    for name, val in davos.core.config.DavosConfig.__dict__.items():
        if isinstance(val, property):
            all_properties.append(name)
    read_only_fields = {
        'conda_avail', 
        'conda_envs_dirs', 
        'environment', 
        'ipython_shell', 
        'smuggled'
    }
    configurable_fields = set(all_properties) - read_only_fields
    configure_func_kwargs = set(inspect.signature(davos.configure).parameters)
    assert not configurable_fields.symmetric_difference(
        configure_func_kwargs
    ), (
        f"configurable fields: {configurable_fields}\ndavos.configure kwargs: "
        f"{configure_func_kwargs}"
    )

In [ ]:
def test_configure_resets_fields_on_fail():
    active_before = davos.config.active
    confirm_install_before = davos.config.confirm_install
    with raises(davos.core.exceptions.DavosConfigError):
        davos.configure(
            active=False, 
            confirm_install=True, 
            suppress_stdout='BAD VALUE'
        )
    assert davos.config.active is active_before
    assert davos.config.confirm_install is confirm_install_before

In [ ]:
run_tests()